###### Função que verifica se há um outro arquivo caracterizado como tabela bronze e, caso haja, mescla o novo dataframe pandas com o antigo retirando as linhas repetidas sobreescrevendo o arquivo anterior.

In [0]:
def spark_df_to_bronze(df, mnt_folder):
    
    df = df.select([df[c].cast('string') for c in df.columns])
    
    try:
        bronze_df = spark.read.parquet(mnt_folder)
        
    except:
        print("Arquivo não encontrado. Novo arquivo adicionado com sucesso.")
        hist_df = df
    
    else:
        print('Histórico encontrado.')
        bronze_df = spark.read.parquet(mnt_folder)
        hist_df = df.union(bronze_df).distinct()
        
    hist_df.write.mode('overwrite').parquet(mnt_folder)


###### Função que verifica se há um outro arquivo caracterizado como tabela prata e, caso haja, mescla o novo dataframe pandas com o antigo retirando as linhas repetidas sobreescrevendo o arquivo anterior.

In [0]:
def spark_df_to_silver(df, mnt_folder):
    
    try:
        silver_df =  spark.read.parquet(mnt_folder)
        
    except:
        print("Arquivo não encontrado. Novo arquivo adicionado com sucesso.")
        hist_df = df
    
    else:
        print('Histórico encontrado.')
        silver_df =  spark.read.parquet(mnt_folder)
        hist_df = df.union(silver_df).distinct()
    
    hist_df.write.mode('overwrite').parquet(mnt_folder)

#### Função de inserção de dados no banco

In [0]:
def insert_a_data_in_db(df, name_table):
    
    try:
        import msal
    except:
        print('Biblioteca não encontrada. Iniciando a instalação.')
        !pip install msal
        import msal
    """
    Função que recebe o nome da tabela como parâmetro, retorna a inserção no banco de dados informado
    insert_table_in_db(name_table_db).
    retorna a inserção no banco de dados informado
    Se a tabela não existir no banco de dados ela será criada,
    caso a tabela já existir ela será substituída e uma nova será criada com os novos dados.
    """
      
    #Autoridade:
    MSAL_authority = 'https://login.windows.net/b4920c82-7581-491a-9dab-cd2ade2f3ebd'
   
    #url do recurso database
    resource_url = 'https://database.windows.net/.default'
   
    # Service Principal Client ID - Criado em secrets no Key Vault
    service_principal_id = dbutils.secrets.get(scope = 'scope-adb-estudos', key = 'app-reg-adb')
       
    # Databricks Service - Criado em secrets no Key Vault
    service_principal_secret = dbutils.secrets.get(scope = 'scope-adb-estudos', key = 'app-user-databricks')
    
   # String (url) de conexão ao Azure SQL Server
    jdbcUrl = f'jdbc:sqlserver://sql-estudo.database.windows.net:1433;database=db-estudos'
    
    #Criando a instância que sera usada durante todo o ciclo de vida do aplicativo.
    instance = msal.ConfidentialClientApplication(service_principal_id, service_principal_secret, MSAL_authority)
   
   #Enviando solicitação ao AAD para obter um token.
    token = instance.acquire_token_for_client(resource_url)
   
   #acesando o token
    access_token = token['access_token']
   
    try:
        table = (df
                 .write
                 .format('jdbc')
                 .mode('overwrite')
                 .option('url',  jdbcUrl)
                 .option('dbtable', name_table)
                 .option('accessToken', access_token)
                 .save())
        # df_pocco.write.format(jdbc)(url, name_table_db)
        print('\n\n Tabela inserida no banco de dados com sucesso! \n\n')
    except pyspark.sql.utils.AnalysisException:
        print(f'Erro! Tabela já existe no banco de dados, renomeie o nome da tabela para proseguir')
        return pyspark.sql.utils.AnalysisException
    
    return table